##### MA755 Machine Learning - Chapter 8. Dimensionality Reduction - 11 Apr 2017 

These notes are based on, and include images from, [_Hands-On Machine Learning with Scikit-Learn and TensorFlow_](http://shop.oreilly.com/product/0636920052289.do)
- by Aurélien Géron
- Published by O'Reilly Media, Inc., 2017

### Load libraries

In [1]:
import numpy             as np
import pickle

In [2]:
import sklearn.metrics         as sk_me
import sklearn.model_selection as sk_ms
import sklearn.datasets        as sk_ds

### Curse of dimensionality 

You can take the Euclidean distance between two rows (data points) in a data set with all numeric variables by treating the numeric features as the coordinates of the data point:
- The maximum distance between two data points (rows) in a data set with `2`   variables is $\sqrt{2}$
- The maximum distance between two data points (rows) in a data set with `100` variables is $\sqrt{100}=10$
- Your data points become sparsely distibuted
- There is little difference in the distances between data points (Wikipedia)

Dimensionality reduction alleviates some of these (and other problems.)

In addition, it is easier to visualize or analyze a dataset with fewer features.

### Main approaches to dimensionality reduction

1. Projection
1. Manifold learning

[draw a picture]

### Projection

Projections are used to reduce the number of (numeric) features in a dataset. 
The goal though is to retain the ability to make correct predictions or gain useful insights from the dataset with this reduced set of variables. 
The procedure is to treat rows as data points and project them into lower dimensional hyperplanes. 
We replace these numeric data points with their coordinates they project onto in these hyperplanes. 

Rows correspond to _data points_ when all features of the dataset/array are numeric. 
This is the case in the `mydata` numpy array.

In [3]:
iris = sk_ds.load_iris()

mydata = iris.data[:,0:3]

mydata.shape

(150, 3)

Each numeric variable is considered a coordinate of the data point. 
Since we have 3 features then each data point has three coordinates and exists in 3-D space. 

In [4]:
mydata[0:2,:]

array([[ 5.1,  3.5,  1.4],
       [ 4.9,  3. ,  1.4]])

We can project these datapoints onto any lower dimensional hyperplane. 

There are only two types of lower dimensional hyperplanes: lines and planes.

When we project a 3-D data point onto a 2-D plane then we:
- associate the 3-D point to the closed point on the 2-D plane
- replace the 3 features of the data point with the 2 features/coordinates of the point in the (2-D) plane

When we project a 3-D data point onto a 1-D line then we:
- associate the 3-D point to the closest point on the 1-D line
- replace the 3 features of the data point with the 1 feature/coordinate of the point in the (1-D) line

We can define a 2-D plane in many ways. One is to fix one coordinate, for instance the third one, as a constant (for example zero.) Projecting the two data points displayed above onto this 2-D plane gives: 

In [5]:
mydata[0:2,0:2]

array([[ 5.1,  3.5],
       [ 4.9,  3. ]])

We can define a 1-D line in many ways. One is to fix two coordinates, for instance the second and third, as constants (for example zero for both.) Projecting the two data points displayed above onto this 1-D line gives: 

In [6]:
mydata[0:2,0:1]

array([[ 5.1],
       [ 4.9]])

The remaining examples will use the MNIST dataset.

In [7]:
mnist = pickle.load(open( "mnist.p", "rb" ))

We'll use a sample of `10,000` rows to avoid problems with Python and the Jupyter kernel. 

In [8]:
sample_ndx = np.random.randint(mnist.data.shape[0],
                               size=10000)
X = mnist.data[sample_ndx,:]
y = mnist.target[sample_ndx]
(X.shape, 
 y.shape
)

((10000, 784), (10000,))

Standardize the sample dataset of independent variables:

In [9]:
X_centered = X - X.mean(axis=0)

### Principal Component Analysis - Projection

https://en.wikipedia.org/wiki/Principal_component_analysis

- Uses an orthogonal transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components
- __The first principal component has the largest possible variance__ and each succeeding component in turn has the highest variance possible under the constraint that it is orthogonal to the preceding components. 
- The resulting vectors are an uncorrelated orthogonal basis set. PCA is sensitive to the relative scaling of the original variables.

[draw on the board]

In [10]:
from sklearn.decomposition import PCA

Create a PCA object to create `10` principal components.

In [11]:
pca = PCA(n_components = 100)

Create `X10D` with the best 10 principal components:

In [12]:
X10D = pca.fit_transform(X_centered)
X10D.shape

(10000, 100)

Display the variance explained by each component:

In [13]:
pca.explained_variance_ratio_

array([ 0.0991279 ,  0.07033546,  0.06186628,  0.05340114,  0.04883757,
        0.04318485,  0.03250653,  0.02873453,  0.02733597,  0.02361584,
        0.02111418,  0.02046112,  0.01744191,  0.01704569,  0.01590814,
        0.01468382,  0.01315014,  0.01290656,  0.01201142,  0.01154248,
        0.01087343,  0.00989359,  0.00963269,  0.00893746,  0.00883667,
        0.00836688,  0.00822974,  0.00783466,  0.00746203,  0.00695514,
        0.00662609,  0.00627007,  0.00605541,  0.00590151,  0.0056966 ,
        0.00554864,  0.00508467,  0.00485173,  0.00480099,  0.0047245 ,
        0.00452729,  0.00447373,  0.0042061 ,  0.00394409,  0.0038295 ,
        0.00374948,  0.003653  ,  0.00354462,  0.00331808,  0.00328857,
        0.00314304,  0.00309604,  0.00301345,  0.00286261,  0.00282931,
        0.00279318,  0.00271388,  0.00253673,  0.00249099,  0.00246821,
        0.00241144,  0.00236821,  0.00228115,  0.00223837,  0.00214532,
        0.00208436,  0.00198464,  0.00194018,  0.00192087,  0.00

Display the variance explained by all requested components (taken together):

In [14]:
sum(pca.explained_variance_ratio_)

0.91577705848156099

Display the cumulative variance explained:

In [15]:
np.cumsum(pca.explained_variance_ratio_)

array([ 0.0991279 ,  0.16946336,  0.23132964,  0.28473078,  0.33356836,
        0.3767532 ,  0.40925974,  0.43799427,  0.46533024,  0.48894608,
        0.51006026,  0.53052138,  0.54796329,  0.56500898,  0.58091712,
        0.59560095,  0.60875108,  0.62165764,  0.63366905,  0.64521154,
        0.65608497,  0.66597856,  0.67561125,  0.68454871,  0.69338538,
        0.70175226,  0.709982  ,  0.71781666,  0.72527869,  0.73223383,
        0.73885992,  0.74512999,  0.7511854 ,  0.75708691,  0.76278351,
        0.76833216,  0.77341683,  0.77826856,  0.78306954,  0.78779405,
        0.79232134,  0.79679507,  0.80100117,  0.80494526,  0.80877476,
        0.81252424,  0.81617724,  0.81972186,  0.82303994,  0.8263285 ,
        0.82947155,  0.83256758,  0.83558103,  0.83844364,  0.84127295,
        0.84406613,  0.84678001,  0.84931675,  0.85180774,  0.85427594,
        0.85668739,  0.8590556 ,  0.86133675,  0.86357512,  0.86572044,
        0.8678048 ,  0.86978944,  0.87172962,  0.87365049,  0.87

You can also request enough principal components to explain 60% of the variance:

In [16]:
pca = PCA(n_components=0.6)
X60P = pca.fit_transform(X_centered)
X60P.shape

(10000, 17)

### Incremental PCA

In [17]:
from sklearn.decomposition import IncrementalPCA

In [45]:
n_batches    = 50
n_components = 100

In [46]:
inc_pca = IncrementalPCA(n_components=n_components)

In [51]:
for X_batch in np.array_split(X_centered, n_batches):
    print(X_batch.shape)

    #inc_pca.partial_fit(X_batch)

(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)
(200, 784)


In [21]:
X10C100B = inc_pca.transform(X_centered)
X10C100B.shape

(10000, 100)

### Randomized PCA

In [22]:
rnd_pca = PCA(n_components=100, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_centered)

### Kernel PCA - Manifold Learning

In [23]:
from sklearn.decomposition import KernelPCA

In [24]:
rbf_pca = KernelPCA(n_components = 2, 
                    kernel       = "rbf", 
                    gamma        = 0.04)

In [25]:
X_fit = rbf_pca.fit_transform(X_centered)

### GridSearch with KernelPCA

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [27]:
clf = Pipeline([
        ("kpca", KernelPCA(n_components=2)),
        ("log_reg", LogisticRegression())
    ])

param_grid = [{
        "kpca__gamma": [0.04],
        "kpca__kernel": ["rbf", "sigmoid"]
    }]

In [28]:
grid_search = GridSearchCV(clf, param_grid, cv=3)

In [29]:
grid_search.fit(X_centered, y)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(steps=[('kpca', KernelPCA(alpha=1.0, coef0=1, copy_X=True, degree=3, eigen_solver='auto',
     fit_inverse_transform=False, gamma=None, kernel='linear',
     kernel_params=None, max_iter=None, n_components=2, n_jobs=1,
     random_state=None, remove_zero_eig=False, tol=0)), ('log_reg', LogisticRegre...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kpca__gamma': [0.04], 'kpca__kernel': ['rbf', 'sigmoid']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [30]:
grid_search.best_params_

{'kpca__gamma': 0.04, 'kpca__kernel': 'sigmoid'}

### Reconstruction

In [54]:
rbf_pca = KernelPCA(n_components = 4, kernel="rbf", gamma=0.0433,
                    fit_inverse_transform=True)

In [55]:
X_reduced = rbf_pca.fit_transform(X_centered)

In [56]:
X_reduced.shape

(10000, 4)

In [33]:
X_preimage = rbf_pca.inverse_transform(X_reduced)

In [53]:
X_preimage.shape

(10000, 784)

In [34]:
from sklearn.metrics import mean_squared_error
mean_squared_error(X_centered, X_preimage)

4355.1372057862372

### Locally Linear Embedding (LLE) - Manifold Learning

- https://www.cs.nyu.edu/~roweis/lle/papers/lleintro.pdf
- https://www.stat.cmu.edu/~cshalizi/350/lectures/14/lecture-14.pdf (TRY THIS ONE)
- http://scikit-learn.org/stable/modules/generated/sklearn.manifold.LocallyLinearEmbedding.html

From the text:
> Here’s how LLE works: first, for each training instance $x(i)$, the algorithm identifies its k closest neighbors (in the preceding code $k = 10$), then tries to reconstruct $x(i)$ as a linear function of these neighbors. More specifically, it finds the weights $w_{i,j}$ such that the squared distance between x(i) and $\sum_{i=1}^{m}w_{i,j}x^{(j)}$ is as small as possible, assuming $w_{i,j} = 0$ if $x(j)$ is not one of the $k$ closest neighbors of $x(i)$. Thus the first step of LLE is the constrained optimization problem described in Equation 8-4, where $W$ is the weight matrix containing all the weights $w_{i,j}$. The second constraint simply normalizes the weights for each training instance $x(i)$.

Simply summarized:

- Create a linear regression for each data point

In [59]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=5, n_neighbors=10)
X_reduced = lle.fit_transform(X_centered)

In [58]:
X_reduced.shape

(10000, 4)

The reconstruction error is the residual sum of square difference between the data points and the linear estimates. 

In [44]:
lle.reconstruction_error_

8.1268809682965764e-07

### The end